In [1]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION'] = spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Install Postgres Driver
!wget https://jdbc.postgresql.org/download/postgresql-42.5.0.jar

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [950 k

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-10 03:32:59--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.94MB/s    in 0.2s    

2022-10-10 03:32:59 (5.94 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Apparel_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Apparel_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   32158956|R1KKOXHNI8MSXU|B01KL6O72Y|      24485154|Easy Tool Stainle...|         Apparel|          4|            0|          0|   N|                Y|★ THESE REALLY DO...|These Really Do W...|2013-01-14 00:00:00|
|         US|    2714559|R26SP2OPDK4HT7|B01ID3ZS5W|     363128556|V28 Women Cowl Ne...| 

In [5]:
# Create DataFrame
review_df = df.filter(df["total_votes"]>= 20)
helpful_votes = review_df.filter(review_df["helpful_votes"]/review_df["total_votes"]>= 0.5)

In [6]:
helpful_votes.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   17162238|R35PT06NWP7LDP|B0140UFQKE|     555753462|Women Padded Spor...|         Apparel|          5|           30|         32|   N|                N|This is an extrem...|This is an extrem...|2015-08-28 00:00:00|
|         US|     453792| RQW4AFOG9MR4Z|B013QTWRAC|     254612789|UONBOX Celebrity ...| 

In [7]:
paid_df = helpful_votes.filter(helpful_votes["vine"]=="Y")

In [8]:
unpaid_df = helpful_votes.filter(helpful_votes["vine"]=="N")

In [9]:
paid_df.describe().show()

+-------+-----------+--------------------+--------------+----------+------------------+--------------------+----------------+------------------+-----------------+-----------------+----+-----------------+--------------------+--------------------+
|summary|marketplace|         customer_id|     review_id|product_id|    product_parent|       product_title|product_category|       star_rating|    helpful_votes|      total_votes|vine|verified_purchase|     review_headline|         review_body|
+-------+-----------+--------------------+--------------+----------+------------------+--------------------+----------------+------------------+-----------------+-----------------+----+-----------------+--------------------+--------------------+
|  count|         33|                  33|            33|        33|                33|                  33|              33|                33|               33|               33|  33|               33|                  33|                  33|
|   mean|       

In [10]:
unpaid_df.describe().show()

+-------+-----------+--------------------+--------------+---------------+--------------------+--------------------+----------------+------------------+------------------+-----------------+-----+-----------------+---------------+--------------------+
|summary|marketplace|         customer_id|     review_id|     product_id|      product_parent|       product_title|product_category|       star_rating|     helpful_votes|      total_votes| vine|verified_purchase|review_headline|         review_body|
+-------+-----------+--------------------+--------------+---------------+--------------------+--------------------+----------------+------------------+------------------+-----------------+-----+-----------------+---------------+--------------------+
|  count|      45388|               45388|         45388|          45388|               45388|               45388|           45388|             45388|             45388|            45388|45388|            45388|          45388|               45246|


In [11]:
paid_number = paid_df.count()
paid_fivestar_number = paid_df[paid_df["star_rating"]==5].count()
paid_fivestar_percentage = paid_fivestar_number/paid_number

In [12]:
print(paid_number)
print(paid_fivestar_number)
print(paid_fivestar_percentage)

33
15
0.45454545454545453


In [13]:
unpaid_number = unpaid_df.count()
unpaid_fivestar_number = unpaid_df[paid_df["star_rating"]==5].count()
unpaid_fivestar_percentage = unpaid_fivestar_number/unpaid_number

In [14]:
print(unpaid_number)
print(unpaid_fivestar_number)
print(unpaid_fivestar_percentage)

45388
23733
0.5228915131752886
